In [1]:
import tarfile
import urllib.request
url = "http://pcai056.informatik.uni-leipzig.de/downloads/corpora/deu_wikipedia_2021_10K.tar.gz"
with urllib.request.urlopen(url) as f:
    stream = urllib.request.urlopen(url)
    f = tarfile.open(fileobj=stream, mode="r|gz")
    f.extractall()


In [2]:
import random
with open("deu_wikipedia_2021_10K/deu_wikipedia_2021_10K-sentences.txt", "r") as f:
    lines = f.read().split("\n")[:-1]
    sents = [l.split("\t")[1] for l in lines]
    random.shuffle(sents)
sents[:5]

['Er war der Sohn des französischen Büchsenmachers Abraham Houel I. (um 1631–1711) und der jüngere Bruder des Hofrüstmeisters Abraham Houel II.',
 'Auch weitere Gründungsmitglieder verließen daraufhin das Projekt.',
 'Malcolm Freshney leitete anfangs das Unternehmen.',
 'Cushing feiert im Jahr 2006 einen Sieg über Stanford.',
 'Ob das Lager auch über quadratische, innen angesetzte Zwischen- und Ecktürme verfügte ist nicht bekannt aber sehr wahrscheinlich.']

In [3]:
import de_dep_news_trf
from itertools import chain
nlp = de_dep_news_trf.load()
def get_nouns(sent):
    doc = nlp(sent)
    return [w for w in doc if w.pos_ == "NOUN"]
nouns = list(chain(*[get_nouns(s) for s in sents[:100]]))


/opt/homebrew/Caskroom/miniconda/base/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [4]:
import re

def is_range(l):
    a = l[0]
    for b in l[1:]:
        if b - a != 1:
            return False
        a = b
    return True

def is_indefinite_nom(t):
    if t.morph.get("Case") == ["Nom"]:
        in_sing = any([c.lemma_ in ["ein", "einen"] for c in t.children])
        in_plur = not any(c.lemma_ in ["der"] for c in t.children)
        return in_sing or in_plur
    return False

def is_article(parent, t):
    return t.head == parent and t.lemma_ in ["der", "ein", "einen"]

def get_phrase(noun):
    if all([t.is_alpha for t in noun.subtree]) \
        and is_indefinite_nom(noun) \
        and is_range(sorted([t.i for t in noun.subtree])):
        ts = [t.text_with_ws for t in noun.subtree if not is_article(noun, t)]
        return re.sub(r"[^a-zäöÜA-ZÄÖÜ]+$", "", "".join(ts))
    return None

phrases = [phrase for noun in nouns if (phrase := get_phrase(noun)) is not None]

In [5]:
phrases[:10]

['Auch weitere Gründungsmitglieder',
 'Ein Arbeitsauftrag für ein größeres Kirchenwerk',
 'Kardinal Giordano',
 'Diese Galerien',
 'nur zwei Parteien',
 'privates Freilichtmuseum mit zahlreichen Bahnartefakten',
 'alle produzierten Wagen',
 'unterschiedliche Schreibungen eines Personennamens',
 'Leitfaden zur sprachlichen Gleichbehandlung von Soldaten',
 'Hoch gelegene Burg']

In [6]:
%load_ext dotenv
%dotenv
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

def replace(a, b, sent):
    response = openai.Edit.create(
    engine="text-davinci-edit-001",
    input=sent,
    instruction=f"Ersetze \"{a}\" durch \"{b}\". Passe die Grammatik entsprechend an.",
    temperature=0,
    top_p=1
    )
    return response["choices"][0]["text"]

In [7]:
import json

random.shuffle(nouns)
random.shuffle(phrases)
data = []
for noun, phrase in list(zip(nouns, phrases)):
    new = replace(noun.text, phrase, noun.doc.text)
    data.append(dict(x=noun.doc.text,replace=noun.text,replace_with=phrase,y=new))
    with open("training_data.json", "w") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)


RateLimitError: Rate limit reached for default-text-davinci-edit-001. Limit: 20.000000 / min. Current: 24.000000 / min. Contact support@openai.com if you continue to have issues.